In [1]:
# 미설치 환경에서는 다음 명령의 실행이 필요합니다.
# %pip install deepface
# %pip install mediapipe
# %conda install -c conda-forge dlib
# %pip install openpyxl

1. 윈도우 기준, `C:/Users/<유저명>/.conda/envs/<conda 환경명>/Lib/site-packages/deepface/DeepFace.py` 674번째 줄에 아래 코드를 추가해야 원활하게 작동합니다.(input shape를 맞춰주기 위한 코드입니다.)
    맥에서도 conda 환경을 모은 곳에서 `Lib/site-packages/deepface/DeepFace.py` 파일을 확인하면 될 것으로 추측됩니다.
    ```
    if model_name == 'DeepID':
        img = np.transpose(img, (0, 2, 1, 3)) 
    ```

2. dlib의 경우 gdown을 통한 데이터 다운로드의 실패하는 경우가 있습니다. 이 경우 가중치 파일을 윈도우 기준으로, 아래 디렉토리에 넣으면 됩니다.
- `C:/Users/<유저명>/.deepface/weights`

    맥의 경우에도 `.deepface/weights`의 형태로 가중치 파일을 다운받는 위치가 있을 것으로 추측됩니다.
- dlib이 사용하는 dat 파일은 https://github.com/ageitgey/face_recognition_models/raw/master/face_recognition_models/models/dlib_face_recognition_resnet_model_v1.dat 를 통해 다운받을 수 있습니다.

In [2]:
import pandas as pd

# 데이터셋 경로입니다. 맞춰주시면 됩니다.
xlsx_path = "AI_DATA_edit_labeled.xlsx"

df = pd.read_excel(xlsx_path)
df

,id,sex,auth,profiles,"labeled_result_value(0: 미인증, 2: 인증)"
0,12,여성,https://sinor.s3.ap-northeast-2.amazonaws.com/...,https://sinor.s3.ap-northeast-2.amazonaws.com/...,0
1,13,여성,https://sinor.s3.ap-northeast-2.amazonaws.com/...,https://sinor.s3.ap-northeast-2.amazonaws.com/...,0
2,11,여성,https://sinor.s3.ap-northeast-2.amazonaws.com/...,https://sinor.s3.ap-northeast-2.amazonaws.com/...,0
3,9,여성,https://sinor.s3.ap-northeast-2.amazonaws.com/...,https://sinor.s3.ap-northeast-2.amazonaws.com/...,0
4,25,남자,https://sinor.s3.ap-northeast-2.amazonaws.com/...,https://sinor.s3.ap-northeast-2.amazonaws.com/...,0
...,...,...,...,...,...
968,2423,남자,https://sinor.s3.ap-northeast-2.amazonaws.com/...,https://sinor.s3.ap-northeast-2.amazonaws.com/...,0
969,2490,남자,https://sinor.s3.ap-northeast-2.amazonaws.com/...,https://sinor.s3.ap-northeast-2.amazonaws.com/...,0
970,2641,남자,https://sinor.s3.ap-northeast-2.amazonaws.com/...,https://sinor.s3.ap-northeast-2.amazonaws.com/...,0
971,2715,남자,https://sinor.s3.ap-northeast-2.amazonaws.com/...,https://sinor.s3.ap-northeast-2.amazonaws.com/...,0


In [3]:
## 수정 필요 ##

# models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
models = ["VGG-Face", "Facenet512", "arcface"] # 이 부분을 두 모델의 리스트로 변경하면 됩니다.
metrics = ["cosine", "euclidean", "euclidean_l2"]

In [2]:
#최대 프로필사진 수 구하기
# print(df['profiles'].apply(lambda x: x.count(',')).max() + 1)

5


In [8]:
# from deepface import DeepFace

# result = DeepFace.verify('../temp/auth_19351675822454273.jpg', '../temp/profiles_1675822458972.jpg', detector_backend='mediapipe', model_name=models[0])

In [4]:
from deepface import DeepFace
import time

def get_verification_result(x, model, distance_metric):
    """
    df row와 모델명을 가지고 DeepFace객체로 verification 결과를 반환함. 순서에 유의할 것.
    """
    images = x['profiles'].split(',')
    source = x['auth']
    
    threshold = 'unknown'

    facial_areas_values = []
    
    time_values = []
    valid_times = []
    average_valid_time = '(No Face)'

    distance_values = []

    verified = 'FALSE'
    verified_values = []
    for imageUrl in images:
        target = imageUrl
        start_time = time.time()
        try:
            verification_dict = DeepFace.verify(source, target, detector_backend='mediapipe', model_name=model, distance_metric= distance_metric)
            threshold = verification_dict['threshold']
            valid_times.append(float(verification_dict['time']))
        except:
            error_time = time.time()
            elapsed_time = round(error_time - start_time, 6)
            verification_dict = {'verified': False,
                'distance': '(No Face)',
                'threshold': 'unknown',
                'model': model,
                'detector_backend': 'mediapipe',
                'similarity_metric': distance_metric,
                'facial_areas': '(No Face)',
                'time': elapsed_time}
        
        facial_areas_values.append(verification_dict['facial_areas'])
        time_values.append(verification_dict['time'])
        distance_values.append(verification_dict['distance'])

        if verification_dict['verified'] == False:
            verified_values.append(0)
        else:
            verified = 'TRUE'
            verified_values.append(1)
    if len(valid_times) != 0:
        average_valid_time = sum(valid_times) / len(valid_times)
    return model, threshold, verification_dict['detector_backend'], distance_metric, facial_areas_values, time_values, average_valid_time, distance_values, verified, verified_values

In [6]:
from tqdm import tqdm

tqdm.pandas()

for model in models:
    df_test = df[0:10].copy()  
    df_test[['model', 'threshold', 'detector_backend', 'similarity_metric', 'facial_areas_values', 'time_values(초)', '평균유효시간', 'distance_values', 'verified', 'verified_values']] = df_test.progress_apply(
        lambda x: pd.Series(get_verification_result(x, model, metrics[0])),  axis=1)

df_test

 50%|█████     | 5/10 [00:05<00:04,  1.25it/s]

In [ ]:
# filename = 'test_deepface_AI_DATA_10ONLY.xlsx'
# df_test.to_excel(filename, index=False)

In [ ]:
# df_verified = df.copy()

# df_verified[['model', 'threshold', 'detector_backend', 'similarity_metric', 'facial_areas_values', 'time_values(초)', '평균유효시간', 'distance_values', 'verified', 'verified_values']] = df.apply(
#     lambda x: pd.Series(get_verification_result(x, 'Dlib')),  axis=1)

In [ ]:
# file_prefix = 'test_deepface_AI_DATA-'
# filename = file_prefix + 'model' + '.xlsx'
# filename

In [15]:
# 데이터프레임 각 행별로 verification함수를 돌린 후 각 모델별 파일명으로 엑셀 익스포트함.
filename = 'test_deepface_AI_DATA_ver2.xlsx'
        
# 엑셀 파일 생성
with pd.ExcelWriter(filename) as writer:
    for model in models:
        for distance_metric in metrics:
            df_verified = df.copy()
            df_verified[['model', 'threshold', 'detector_backend', 'similarity_metric', 'facial_areas_values', 'time_values(초)', '평균유효시간', 'distance_values', 'verified', 'verified_values']] = df.progress_apply(
                lambda x: pd.Series(get_verification_result(x, model)),  axis=1)
            
            sheet_name = model + f'({distance_metric})'
            
            # 각 데이터프레임을 엑셀 시트에 쓰기
            df_verified.to_excel(writer, sheet_name=sheet_name, index=False)